# Generating LLM client

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

# Using Images

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
import base64

with open("data/cat.jpg", "rb") as f:
    image_data = base64.b64encode(f.read()).decode()

message = HumanMessage(
    content=[
        {"type": "text", "text": "describe the image"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
)
response = llm.invoke([message])
print(response.content)

## Extracting structured information

In [ ]:
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="first name of a Person")
    lastname: str = Field(description="last name of a Person")
    email: str = Field(description="Email of a Person")
    age: int = Field(description="Age of a Person")


response = llm.with_structured_output(Person).invoke([
    ("system", "you are an specialist extracting person information from a text"),
    ("user", "mi nombre es james espichan, mi corre es james@idat.com y tengo 50 años")
])

In [ ]:
response

## Let's use a bill

In [ ]:
class Item(BaseModel):
    description: str = Field(description="description for each item")
    price: float = Field(description="price of each item")

class Bill(BaseModel):
    items: list[Item] = Field(description="List of item that contains descriptions for each item and prices")

In [ ]:
with open("data/factura1.png", "rb") as f:
    image_data = base64.b64encode(f.read()).decode()

In [ ]:
response = llm.with_structured_output(Bill).invoke([
    ("system", "you are an specialist extracting items information from a billing"),
    HumanMessage(content=[
        {"type": "text", "text": "billing"},
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ],
    )
])

In [ ]:
response.items